In [20]:
import pandas as pd
import numpy as np
from os import path, listdir

In [2]:
dr1=r"C:\Users\vadim\Desktop\fourth\correl"
dr2=r"C:\Users\vadim\Desktop\fourth"
fn2=r"температура.xlsx"
fn1=r"станции по бассейнам.xlsx"
fn3=r"станции ЕЧР1.xlsx"

In [3]:
np.seterr(invalid='ignore')
np.set_printoptions(precision=3, suppress=True)
dtr=np.pi/180.0#degrees_to_radians

In [5]:
ind_st=pd.read_excel(path.join(dr2,fn1),sheet_name=0,header=0,index_col=0)
ind=ind_st.index.unique()
coor_st=pd.read_excel(path.join(dr2,fn3),sheet_name=0,header=0,index_col=0)
err=pd.read_excel(path.join(dr2,fn2),sheet_name=0,header=0,index_col=0)

In [15]:
for i in ind:
    stn=pd.Index(ind_st["index_st"].loc[i]).intersection(err.columns)
    coor=coor_st.loc[stn, ["lat_1","lon_1"]]
    lat=np.array(coor["lat_1"])*dtr
    lon=np.array(coor["lon_1"])*dtr
    lon1, lon2 = np.meshgrid(lon, lon)
    lon_diff = lon1 - lon2
    lat1, lat2 = np.meshgrid(lat, lat)
    angle = (np.sin((lat2-lat1)/2)**2+np.cos(lat1)*np.cos(lat2)*np.sin(lon_diff/2)**2)**0.5
    arc = 6371*2*np.arcsin(angle)
    
    cor=err.loc[:,stn].corr().values
    arc=arc.ravel()
    cor=cor.ravel()
    a=np.vstack((arc,cor))
    a=pd.DataFrame(a.transpose(), columns=["расстояние", "корреляция"])
    a=a[a["расстояние"]>1]
    a.to_excel(path.join(dr1,i+"_temp"+".xlsx"))

In [17]:
from scipy.optimize import curve_fit

In [18]:
def func2(x, a, b):
    return a-x/b

In [21]:
dr=listdir(dr1)
fns=[x[:-5] for x in dr]

In [22]:
res=[]
for i in dr:
    print(i)
    fr=pd.read_excel(path.join(dr1,i),sheet_name=0,header=0,index_col=0)
    fr = fr[fr["расстояние"]< 200].sort_values(by=["расстояние"]).values[::2]
    fr[:,1]=0.5*np.log((fr[:,1]+1)/(1-fr[:,1]))
    k=4*(fr.shape[0]//4)
    fr=np.mean(np.reshape(fr[:k],(-1,4,2)), axis=1)
    fr[:,1]=(np.exp(2*fr[:,1])-1)/(np.exp(2*fr[:,1])+1)
    fr[fr<0] = 0
    popt, pcov = curve_fit(func2, fr[:,0], fr[:,1],bounds=([0.5,1], [1., 300]))
    res.append(popt)

Дон-Беляевский_temp.xlsx
Ока_temp.xlsx
Печора_temp.xlsx
Северная Двина_temp.xlsx


In [23]:
res=pd.DataFrame(np.stack(res), index=fns, columns=[r"r(0)/a",r"l0"])

In [24]:
res.to_excel(path.join(dr2,"curve_parameter_stations1.xlsx"))